In [1]:
#https://www.kaggle.com/anikannal/solar-power-generation-data/tasks
#https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/discussion/44629

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.metrics import median_absolute_error, mean_squared_error
from sklearn.linear_model import Ridge
from category_encoders import OneHotEncoder
from sklearn.pipeline import make_pipeline

In [2]:
train = pd.read_parquet("train.parquet")
val = pd.read_parquet("val.parquet")

In [3]:
train2 = train.copy()
train2['WEEKDAY'] = train2['DATE_TIME'].dt.weekday
train2['HOUR'] = train2['DATE_TIME'].dt.hour
train2['MINUTE'] = train2['DATE_TIME'].dt.minute


val2 = val.copy()
val2['WEEKDAY'] = val2['DATE_TIME'].dt.weekday
val2['HOUR'] = val2['DATE_TIME'].dt.hour
val2['MINUTE'] = val2['DATE_TIME'].dt.minute

cats = ['SOURCE_KEY', 'WEEKDAY','HOUR', 'MINUTE']

In [4]:

encoder = OneHotEncoder(cols=cats, use_cat_names=True)
train2_ohe = encoder.fit_transform(train2[cats])
val2_ohe = encoder.transform(val2[cats])

In [5]:
import torch.nn as nn
import torch
import torch.optim as opt
import torch.nn.functional as F

In [6]:
train2_ohe.shape

(33112, 57)

# Denoising Autoencoder

In [7]:
class DenoisingAE(nn.Module):
    def __init__(self):
        super(DenoisingAE, self).__init__()

        self.hid1 = nn.Linear(57, 100)
        self.hid2 = nn.Linear(100, 20)
        self.hid3 = nn.Linear(20, 100)
        self.out = nn.Linear(100, 57)
    
    def forward(self, x, return_embeddings=False):
        x = F.relu(self.hid1(x))
        x = self.hid2(x)
        
        if return_embeddings:
            return x
        
        x = F.relu(self.hid3(x))
        x = self.out(x)
        

        return x
        

In [8]:
import numba
from numba import float64
from random import random

In [9]:
@numba.vectorize([float64(float64, float64)], target='parallel')
def flip(x, p=0.5):
    if random() < p:
        if x == 1:
            return 0
        else: 
            return 1
    return x
    
#flip_np = np.vectorize(flip)
flip_np = flip

In [16]:
flip(np.array([1,1,1]), 0.5)

array([0., 0., 0.])

In [11]:
torch.manual_seed(0)
net = DenoisingAE().cuda()
criterion = nn.MSELoss()
optimizer = opt.Adam(net.parameters(), lr=1e-4)


x_true = torch.from_numpy(train2_ohe.values).cuda().float()
for e in range(10000):
    optimizer.zero_grad()
    x_noisy = flip_np(train2_ohe.values, 0.1) #certamente tem uma forma melhor de fazer isso
    x_noisy = torch.from_numpy(x_noisy).cuda().float()
    
    p = net(x_noisy).cuda()
    loss = criterion(p, x_true)
    #print(loss.item())
    
    loss.backward()
    optimizer.step()
    
    p_numpy = p.cpu().detach().numpy()
    if e % 500 == 0:
        print(np.sqrt(mean_squared_error(train2_ohe, p_numpy)))
        

0.27475878137960985
0.2204230850457685
0.20913584099173363
0.20813995932223742
0.2062300138906113
0.205191300383556
0.20432007897884852
0.20387374602620073
0.20321670821477278
0.2026125293335492
0.20202511478757632
0.20148881446431557
0.20107313212651054
0.20074950346510284
0.2005095032811797
0.20041812575138984
0.20031038094620096
0.19992555141798163
0.1996446220797017
0.19921921522276897


# Embedding to Tree

In [12]:
from sklearn.ensemble import RandomForestRegressor
mdl = RandomForestRegressor(n_jobs=-1, random_state=0, n_estimators=100)

x = torch.from_numpy(train2_ohe.values).cuda().float()
Xtr = net(x,return_embeddings=True).detach().cpu().numpy()
ytr = train2['Y4WIN']
mdl.fit(Xtr,ytr)

x = torch.from_numpy(val2_ohe.values).cuda().float()
Xval =  net(x,return_embeddings=True).detach().cpu().numpy()

p = mdl.predict(Xval)
median_absolute_error(val2['Y4'], p)

9.02944940716667